# LangChain으로 RAG 챗봇 만들기: 생각보다 쉽습니다!

RAG(Retrieval-Augmented Generation)라는 용어를 들으면 복잡한 기술처럼 느껴지시나요? 오늘은 LangChain을 사용해서 **30줄도 안 되는 코드**로 RAG 기반 챗봇을 만드는 방법을 소개합니다.

---

## 목차

1. [RAG란 무엇인가?](#1-rag란-무엇인가)
2. [필요한 도구들](#2-필요한-도구들)
3. [Vector Store 연결하기](#3-vector-store-연결하기)
4. [Temperature 파라미터 이해하기](#4-temperature-파라미터-이해하기)
5. [RAG 파이프라인 구현](#5-rag-파이프라인-구현)
6. [Gradio로 웹 UI 만들기](#6-gradio로-웹-ui-만들기)
7. [마무리](#7-마무리)

---

## 1. RAG란 무엇인가?

**RAG (Retrieval-Augmented Generation)** 는 LLM의 한계를 극복하기 위한 기술입니다.

### LLM의 한계

| 문제 | 설명 |
|------|------|
| 지식 차단(Knowledge Cutoff) | 학습 데이터 시점 이후의 정보를 모름 |
| 환각(Hallucination) | 모르는 내용을 그럴듯하게 지어냄 |
| 도메인 지식 부족 | 특정 회사/제품의 내부 정보를 알 수 없음 |

### RAG의 해결책

```
사용자 질문 → 관련 문서 검색 → 문서 + 질문을 LLM에 전달 → 정확한 답변
```

외부 지식베이스에서 관련 정보를 **검색(Retrieval)** 한 후, 이를 바탕으로 LLM이 답변을 **생성(Generation)** 합니다.

## 2. 필요한 도구들

이번 튜토리얼에서 사용할 라이브러리들입니다:

| 라이브러리 | 역할 |
|-----------|------|
| `langchain` | RAG 파이프라인 프레임워크 |
| `langchain-openai` | OpenAI LLM 연동 |
| `langchain-chroma` | ChromaDB 벡터 스토어 연동 |
| `langchain-huggingface` | HuggingFace 임베딩 모델 사용 |
| `gradio` | 웹 UI 생성 |

In [ ]:
# 필요한 패키지 설치 (필요시 주석 해제)
# !pip install langchain langchain-openai langchain-chroma langchain-huggingface gradio python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage

import gradio as gr

# 환경변수 로드 (.env 파일에 OPENAI_API_KEY 필요)
load_dotenv(override=True)
print("환경 설정 완료!")

## 3. Vector Store 연결하기

RAG의 핵심은 **Vector Store**입니다. 문서들을 벡터(숫자 배열)로 변환해서 저장해두면, 질문과 의미적으로 유사한 문서를 빠르게 찾을 수 있습니다.

### 임베딩 모델 선택

HuggingFace의 `sentence-transformers/all-MiniLM-L6-v2` 모델을 사용합니다:
- 무료로 사용 가능
- 로컬에서 실행 (API 비용 없음)
- 빠른 속도와 준수한 성능

In [ ]:
# 설정값 정의
LLM_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
CHROMA_PATH = "./chroma_db"

# 임베딩 모델 초기화
embed_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

# ChromaDB 연결
vector_db = Chroma(
    persist_directory=CHROMA_PATH,
    embedding_function=embed_model
)

print(f"Vector DB 연결 완료: {CHROMA_PATH}")
print(f"저장된 문서 수: {vector_db._collection.count()}")

## 4. Temperature 파라미터 이해하기

LLM을 사용할 때 자주 보는 `temperature` 파라미터에 대해 알아봅시다.

### Temperature란?

흔히 "창의성"을 조절하는 파라미터라고 설명하지만, 정확히는 **토큰 선택의 다양성**을 조절합니다.

| Temperature | 동작 | 적합한 용도 |
|-------------|------|------------|
| 0 | 항상 가장 확률 높은 토큰 선택 | 사실 기반 Q&A, 코드 생성 |
| 0.5 | 중간 정도의 다양성 | 일반 대화 |
| 1.0 | 확률에 비례하여 토큰 선택 | 창의적 글쓰기 |

### 주의사항

- `temperature=0`이라고 해서 항상 동일한 결과가 나오지는 않습니다
- 완전한 재현성을 원하면 `seed` 파라미터도 함께 설정해야 합니다
- 창의적인 답변을 원한다면 temperature보다 **System Prompt**를 활용하는 것이 효과적입니다

In [ ]:
# Retriever 생성: 유사 문서를 검색하는 객체
doc_retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # 상위 4개 문서 반환
)

# LLM 생성: 답변을 생성하는 객체
chat_model = ChatOpenAI(
    model=LLM_MODEL,
    temperature=0  # 일관된 답변을 위해 0으로 설정
)

print("Retriever와 LLM 준비 완료!")

### Retriever와 LLM 테스트

LangChain 객체들은 `invoke()` 메서드로 실행합니다.

In [ ]:
# Retriever 테스트: 관련 문서 검색
sample_query = "서비스 이용 방법을 알려줘"
retrieved_docs = doc_retriever.invoke(sample_query)

print(f"검색 쿼리: {sample_query}")
print(f"검색된 문서 수: {len(retrieved_docs)}\n")

for idx, doc in enumerate(retrieved_docs, 1):
    print(f"--- 문서 {idx} ---")
    print(doc.page_content[:150] + "...\n")

In [ ]:
# LLM 테스트: 직접 질문 (RAG 없이)
direct_response = chat_model.invoke("안녕하세요, 반갑습니다!")
print(f"LLM 응답: {direct_response.content}")

## 5. RAG 파이프라인 구현

이제 모든 조각을 합쳐서 RAG 파이프라인을 만들어봅시다.

### RAG 파이프라인 흐름

```
1. 사용자 질문 입력
       ↓
2. Retriever로 관련 문서 검색
       ↓
3. 검색된 문서를 Context로 구성
       ↓
4. System Prompt + Context + 질문을 LLM에 전달
       ↓
5. LLM이 문서를 참고하여 답변 생성
```

In [ ]:
# System Prompt 정의
ASSISTANT_PROMPT = """
당신은 TechStore의 고객 지원 전문가입니다.
고객의 질문에 친절하고 정확하게 답변해주세요.

아래 참고 자료를 활용하여 답변하되, 참고 자료에 없는 내용은 
"해당 정보는 확인이 필요합니다"라고 안내해주세요.

참고 자료:
{references}
"""

In [ ]:
def get_rag_response(user_query: str, chat_history: list) -> str:
    """
    RAG 파이프라인으로 사용자 질문에 답변합니다.
    
    Args:
        user_query: 사용자 질문
        chat_history: 이전 대화 내역 (Gradio용)
    
    Returns:
        AI 응답 텍스트
    """
    # 1단계: 관련 문서 검색
    relevant_docs = doc_retriever.invoke(user_query)
    
    # 2단계: 검색 결과를 하나의 텍스트로 합치기
    combined_context = "\n---\n".join(
        doc.page_content for doc in relevant_docs
    )
    
    # 3단계: 프롬프트 구성
    system_content = ASSISTANT_PROMPT.format(references=combined_context)
    
    # 4단계: LLM 호출 및 응답 반환
    messages = [
        SystemMessage(content=system_content),
        HumanMessage(content=user_query)
    ]
    
    ai_response = chat_model.invoke(messages)
    return ai_response.content

In [ ]:
# RAG 파이프라인 테스트
test_query = "반품은 어떻게 하나요?"
response = get_rag_response(test_query, [])

print(f"질문: {test_query}")
print(f"\n답변:\n{response}")

## 6. Gradio로 웹 UI 만들기

Gradio를 사용하면 **단 한 줄**로 채팅 인터페이스를 만들 수 있습니다!

In [ ]:
# 채팅 UI 생성 및 실행
chat_ui = gr.ChatInterface(
    fn=get_rag_response,
    title="TechStore 고객 지원 챗봇",
    description="제품 및 서비스에 대해 무엇이든 물어보세요!",
    examples=["배송은 얼마나 걸리나요?", "환불 정책이 어떻게 되나요?", "회원 가입 방법을 알려주세요"]
)

chat_ui.launch()

실행하면 브라우저에서 채팅 인터페이스가 열립니다. 질문을 입력하면 RAG 파이프라인이 동작하여 Vector Store의 문서를 참고한 답변을 받을 수 있습니다.

## 7. 마무리

### 전체 코드 요약

RAG 챗봇의 핵심은 간단합니다:

```python
# 1. 컴포넌트 초기화
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma(persist_directory="./chroma_db", embedding_function=embed_model)
doc_retriever = vector_db.as_retriever()
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. RAG 함수
def get_rag_response(user_query, chat_history):
    relevant_docs = doc_retriever.invoke(user_query)
    combined_context = "\n---\n".join(doc.page_content for doc in relevant_docs)
    system_content = ASSISTANT_PROMPT.format(references=combined_context)
    messages = [SystemMessage(content=system_content), HumanMessage(content=user_query)]
    return chat_model.invoke(messages).content

# 3. UI 실행
gr.ChatInterface(fn=get_rag_response).launch()
```

### 핵심 구성요소

| 구성요소 | 역할 |
|---------|------|
| Vector Store | 문서를 벡터로 저장하고 유사 문서 검색 |
| Retriever | 질문과 관련된 문서를 찾아오는 역할 |
| LLM | 검색된 문서를 바탕으로 답변 생성 |
| System Prompt | LLM의 역할 정의 및 Context 전달 |
| Gradio | 웹 채팅 UI 제공 |

### 더 발전시키려면?

1. **청킹 전략 개선**: 문서를 의미 단위로 효과적으로 분할
2. **하이브리드 검색**: BM25(키워드) + 벡터 검색 조합
3. **Re-ranking**: Cross-encoder로 검색 결과 재정렬
4. **대화 메모리**: 이전 대화를 기억하는 멀티턴 지원
5. **스트리밍**: 답변을 실시간으로 출력

---

> **RAG가 어렵게 느껴지셨나요?**  
> LangChain 덕분에 복잡한 부분은 추상화되어 있어서, 핵심 로직에만 집중할 수 있습니다.